In [ ]:
import os
import sys

If you use yaml file to set parameters, use below cell as code.

import yaml
conf_file = "conf.yaml"
with open(conf_file, "r") as f:
    conf = yaml.safe_load(f)

If you don't use yaml file to set parameters, use below.


|  key  |  description  |
| :---: | :--- |
|use_devices | use gpu number. If you want to do distributed learning, write like "0,1,2" |
|model_dir |all outputs are written here. <br>e.g., trained model, inference results, etc..|
| train_x_dirs<br> valid_x_dirs<br> test_x_dirs | list of directories where input images exist.|
|  train_y_dirs<br> valid_y_dirs                   | list of directories where segmentation image exsit.<br> Each segmentation image name must be the same for corresponding input image.  |
|which_to_inference <br> which_to_visualise          | chosse from "train", "valid", "test".|
|output_activation                                   | chosse "softmax" or "sigmoid". <br>softmax means each pixcel is assigned to 1 label.<br>sigmoid means each pixel can assigned 1 or more labels.|
|batch_size | batch size|
|n_epochs   |nuber of epochs |
|image_size | [height, width] |
|optimizer  |"Adam" or "Nadam" or "SGD" |
|loss       |choose one .<br>"CE": cross entropy <br> "FL": focal loss <br>"GDL": generalized dice loss
|metrics| choose one. <br> IoU: Intersection over Union. Jaccard index. <br>F1score: F1 score. F-measure. Dice coefficient.|
|check_categorical_metrics| True, if you want to check metrics for each label.|
|use_tensorboard| if you want to use, True. If not, False.|


In [ ]:
conf = dict()
conf["use_devices"] = "0"

conf["model_dir"] = "path/to/output_dir"
conf["label_file_path"] = "path/to/label_list.csv"

conf["train_x_dirs"] = ["path/to/train_x1", "path/to/train_x2"]
conf["train_y_dirs"] = ["path/to/train_y1", "path/to/train_y2"]

conf["valid_x_dirs"] = ["path/to/valid_x1", "path/to/valid_x2"]
conf["valid_y_dirs"] = ["path/to/valid_y1", "path/to/valid_y2"]


conf["test_x_dirs"] = ["path/to/test_x1", "path/to/test_x2"] # if you don't have test data, enter empty list.([])
conf["which_to_inference"] = ["train", "valid", "test"]
# below lines are not needed in training phase.
# conf["which_to_visualise"] = ["valid", "test"]

# conf["output_activation"] = "softmax"

# conf["batch_size"] = 8
# conf["n_epochs"] = 1000
# conf["image_size"] = [512, 512]
# conf["optimzer"] = "Adam"
# conf["loss"] = "FL"
# conf["use_tensorboard"] = False


In [ ]:
use_devices = str(conf["use_devices"])

model_dir = conf["model_dir"]
label_file_path = conf["label_file_path"]

batch_size = conf["batch_size"]
image_size = conf["image_size"]


train_x_dirs = conf["train_x_dirs"]
train_y_dirs = conf["train_y_dirs"]

valid_x_dirs = conf["valid_x_dirs"]
valid_y_dirs = conf["valid_y_dirs"]

test_x_dirs = conf["test_x_dirs"]

gpu setting

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = use_devices
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

import modules

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from tqdm import tqdm

from deeplab_v3plus_tfkeras.data_utils import make_xy_from_data_paths
from deeplab_v3plus_tfkeras.data_utils import inference_large_img
from deeplab_v3plus_tfkeras.data_utils import save_inference_results
from deeplab_v3plus_tfkeras.input_data_processing import make_xy_path_list
from deeplab_v3plus_tfkeras.label import Label

some settings

In [ ]:
which_to_inference = conf["which_to_inference"]
label = Label(label_file_path)
n_gpus = len(use_devices.split(','))


batch_size = batch_size * n_gpus

model_file = os.path.join(model_dir, 'best_model.h5')

load model

In [ ]:
model_file = os.path.join(model_dir, 'best_model.h5')
if n_gpus >= 2:
    strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():
        model = keras.models.load_model(model_file, compile=False)

else:
    model = keras.models.load_model(model_file, compile=False)

model.summary()

preprocess = keras.applications.xception.preprocess_input
last_activation = model.layers[-1].name

inference training dataset

In [ ]:
if "train" in which_to_inference:
    x_paths, y_paths = make_xy_path_list(train_x_dirs, train_y_dirs)
    x, y = make_xy_from_data_paths(x_paths,
                                   y_paths,
                                   image_size,
                                   label,
                                   resize_or_crop="crop")
    pred = model.predict(preprocess(x), batch_size=batch_size)
    fpath = os.path.join(model_dir, "train_inference.h5")
    save_inference_results(fpath,
                           x=x,
                           y=y,
                           pred=pred,
                           last_activation=last_activation)

inference validation dataset

In [ ]:
if "valid" in which_to_inference:
    x_paths, y_paths = make_xy_path_list(valid_x_dirs, valid_y_dirs)
    x, y = make_xy_from_data_paths(x_paths,
                                   y_paths,
                                   image_size,
                                   label,
                                   resize_or_crop="crop")
    pred = model.predict(preprocess(x), batch_size=batch_size)
    fpath = os.path.join(model_dir, "valid_inference.h5")
    save_inference_results(fpath,
                           x=x,
                           y=y,
                           pred=pred,
                           last_activation=last_activation)

inference test dataset.

In [ ]:
if "test" in which_to_inference:
    for i, test_data_dir in enumerate(test_x_dirs):
        test_name = test_data_dir.split(os.sep)[-1]
        x_paths, _ = make_xy_path_list([test_data_dir], None)

        mode = "max_confidence"
        x = []
        pred = []
        for x_path in tqdm(x_paths):
            x0, pred0 = inference_large_img(
                x_path,
                model,
                preprocess,
                mode=mode,
                threshold=0.5,
                batch_size=batch_size)
            x.append(x0)
            pred.append(pred0)

        x = np.array(x)
        pred = np.array(pred)

        fpath = os.path.join(model_dir, "test_" + test_name + "_inference.h5")
        save_inference_results(fpath,
                               x=x,
                               pred=pred,
                               last_activation=last_activation)
